# Bootstrap the client with ROOT credentials
Using the python client generated from our OpenAPI spec, we generate a token from our root user's credentials

In [3]:
from polaris.catalog.api.iceberg_catalog_api import IcebergCatalogAPI
from polaris.catalog.api.iceberg_o_auth2_api import IcebergOAuth2API
from polaris.catalog.api_client import ApiClient as CatalogApiClient
from polaris.catalog.api_client import Configuration as CatalogApiClientConfiguration

# (CHANGE ME): This credential changes on every Polaris service restart
# In the Polaris log, look for the `realm: default-realm root principal credentials:` string
polaris_credential = 'a59a7c1b69d60701:97fe201482658c0bafa6dc80c7ad13da' # pragma: allowlist secret
client_id, client_secret = polaris_credential.split(":")
client = CatalogApiClient(CatalogApiClientConfiguration(username=client_id,
                                 password=client_secret,
                                 host='http://polaris:8181/api/catalog'))

oauth_api = IcebergOAuth2API(client)
token = oauth_api.get_token(scope='PRINCIPAL_ROLE:ALL',
                            client_id=client_id,
                          client_secret=client_secret,
                          grant_type='client_credentials',
                          _headers={'realm': 'default-realm'})
namespace='ForeignTableDemo'
tableLocation='file:///tmp/polaris/' + namespace
tableName = 'tbl1'


# Create our first catalog

* Creates a catalog named `polaris_catalog` that writes to a specified location in the Local Filesystem.

In [4]:
from polaris.management import *

client = ApiClient(Configuration(access_token=token.access_token,
                                   host='http://polaris:8181/api/management/v1'))
root_client = PolarisDefaultApi(client)

storage_conf = FileStorageConfigInfo(storage_type="FILE", allowed_locations=["file:///tmp", tableLocation])
catalog_name = 'polaris_demo'
catalog = Catalog(name=catalog_name, type='INTERNAL', properties={"default-base-location": tableLocation},
                storage_config_info=storage_conf)
catalog.storage_config_info = storage_conf
root_client.create_catalog(create_catalog_request=CreateCatalogRequest(catalog=catalog))
resp = root_client.get_catalog(catalog_name=catalog.name)
resp

PolarisCatalog(type='INTERNAL', name='polaris_demo', properties=CatalogProperties(default_base_location='file:///tmp/polaris/ForeignTableDemo', additional_properties={}), create_timestamp=1734107895516, last_update_timestamp=1734107895516, entity_version=1, storage_config_info=FileStorageConfigInfo(storage_type='FILE', allowed_locations=['file:///tmp/polaris/ForeignTableDemo', 'file:///tmp']))

# Utility Functions

In [5]:
# Creates a principal with the given name
def create_principal(api, principal_name):
  principal = Principal(name=principal_name, type="SERVICE")
  try:
    principal_result = api.create_principal(CreatePrincipalRequest(principal=principal))
    return principal_result
  except ApiException as e:
    if e.status == 409:
      return api.rotate_credentials(principal_name=principal_name)
    else:
      raise e

# Create a catalog role with the given name
def create_catalog_role(api, catalog, role_name):
  catalog_role = CatalogRole(name=role_name)
  try:
    api.create_catalog_role(catalog_name=catalog.name, create_catalog_role_request=CreateCatalogRoleRequest(catalog_role=catalog_role))
    return api.get_catalog_role(catalog_name=catalog.name, catalog_role_name=role_name)
  except ApiException as e:
    return api.get_catalog_role(catalog_name=catalog.name, catalog_role_name=role_name)
  else:
    raise e

# Create a principal role with the given name
def create_principal_role(api, role_name):
  principal_role = PrincipalRole(name=role_name)
  try:
    api.create_principal_role(CreatePrincipalRoleRequest(principal_role=principal_role))
    return api.get_principal_role(principal_role_name=role_name)
  except ApiException as e:
    return api.get_principal_role(principal_role_name=role_name)


# Create a new Principal, Principal Role, and Catalog Role
The new Principal belongs to the `engineer` principal role, which has `CATALOG_MANAGE_CONTENT` privileges on the `polaris_catalog`. 


`CATALOG_MANAGE_CONTENT` has create/list/read/write privileges on all entities within the catalog. The same privilege could be granted to a namespace, in which case, the engineers could create/list/read/write any entity under that namespace

In [6]:
# Create the engineer_principal
engineer_principal = create_principal(root_client, "collado")

# Create the principal role
engineer_role = create_principal_role(root_client, "engineer")

# Create the catalog role
manager_catalog_role = create_catalog_role(root_client, catalog, "manage_catalog")

# Grant the catalog role to the principal role
# All principals in the principal role have the catalog role's privileges
root_client.assign_catalog_role_to_principal_role(principal_role_name=engineer_role.name,
                                                  catalog_name=catalog.name,
                                                  grant_catalog_role_request=GrantCatalogRoleRequest(catalog_role=manager_catalog_role))

# Assign privileges to the catalog role
# Here, we grant CATALOG_MANAGE_CONTENT
root_client.add_grant_to_catalog_role(catalog.name, manager_catalog_role.name,
                                      AddGrantRequest(grant=CatalogGrant(catalog_name=catalog.name,
                                                                       type='catalog',
                                                                       privilege=CatalogPrivilege.CATALOG_MANAGE_CONTENT)))

# Assign the principal role to the principal
root_client.assign_principal_role(engineer_principal.principal.name, grant_principal_role_request=GrantPrincipalRoleRequest(principal_role=engineer_role))

# Create a reader Principal, Principal Role, and Catalog Role
This new principal belongs to the `product_manager` principal role, which is explicitly granted read and list permissions on the catalog.

Permissions cascade, so permissions granted at the catalog level are inherited by namespaces and tables within the catalog.

In [7]:
# Create a reader principal
reader_principal = create_principal(root_client, "mlee")

# Create the principal role
pm_role = create_principal_role(root_client, "product_manager")

# Create the catalog role
read_only_role = create_catalog_role(root_client, catalog, "read_only")

# Grant the catalog role to the principal role
root_client.assign_catalog_role_to_principal_role(principal_role_name=pm_role.name,
                                                  catalog_name=catalog.name,
                                                  grant_catalog_role_request=GrantCatalogRoleRequest(catalog_role=read_only_role))

# Assign privileges to the catalog role
# Here, the catalog role is granted READ and LIST privileges at the catalog level
# Privileges cascade down
root_client.add_grant_to_catalog_role(catalog.name, read_only_role.name,
                                      AddGrantRequest(grant=CatalogGrant(catalog_name=catalog.name,
                                                                       type='catalog',
                                                                       privilege=CatalogPrivilege.TABLE_LIST)))
root_client.add_grant_to_catalog_role(catalog.name, read_only_role.name,
                                      AddGrantRequest(grant=CatalogGrant(catalog_name=catalog.name,
                                                                       type='catalog',
                                                                       privilege=CatalogPrivilege.TABLE_READ_PROPERTIES)))
root_client.add_grant_to_catalog_role(catalog.name, read_only_role.name,
                                      AddGrantRequest(grant=CatalogGrant(catalog_name=catalog.name,
                                                                       type='catalog',
                                                                       privilege=CatalogPrivilege.TABLE_READ_DATA)))
root_client.add_grant_to_catalog_role(catalog.name, read_only_role.name,
                                      AddGrantRequest(grant=CatalogGrant(catalog_name=catalog.name,
                                                                       type='catalog',
                                                                       privilege=CatalogPrivilege.VIEW_LIST)))
root_client.add_grant_to_catalog_role(catalog.name, read_only_role.name,
                                      AddGrantRequest(grant=CatalogGrant(catalog_name=catalog.name,
                                                                       type='catalog',
                                                                       privilege=CatalogPrivilege.VIEW_READ_PROPERTIES)))
root_client.add_grant_to_catalog_role(catalog.name, read_only_role.name,
                                      AddGrantRequest(grant=CatalogGrant(catalog_name=catalog.name,
                                                                       type='catalog',
                                                                       privilege=CatalogPrivilege.NAMESPACE_READ_PROPERTIES)))
root_client.add_grant_to_catalog_role(catalog.name, read_only_role.name,
                                      AddGrantRequest(grant=CatalogGrant(catalog_name=catalog.name,
                                                                       type='catalog',
                                                                       privilege=CatalogPrivilege.NAMESPACE_LIST)))

# Assign the principal role to the principal
root_client.assign_principal_role(reader_principal.principal.name, grant_principal_role_request=GrantPrincipalRoleRequest(principal_role=pm_role))

# Create a Spark session with the engineer credentials

* Catalog URI points to our Polaris installation
* Credential set using the client_id and client_secret generated for the principal
* Scope set to `PRINCIPAL_ROLE:ALL`
* `X-Iceberg-Access-Delegation` is set to vended-credentials

In [8]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
  .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
  .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.7.1,org.apache.hadoop:hadoop-aws:3.4.0,software.amazon.awssdk:bundle:2.23.19,software.amazon.awssdk:url-connection-client:2.23.19")
  .config('spark.sql.iceberg.vectorization.enabled', 'false')
         
  # Configure the 'polaris' catalog as an Iceberg rest catalog
  .config("spark.sql.catalog.polaris.type", "rest")
  .config("spark.sql.catalog.polaris", "org.apache.iceberg.spark.SparkCatalog")
  # Specify the rest catalog endpoint       
  .config("spark.sql.catalog.polaris.uri", "http://polaris:8181/api/catalog")
  # Enable token refresh
  .config("spark.sql.catalog.polaris.token-refresh-enabled", "true")
  # specify the client_id:client_secret pair
  .config("spark.sql.catalog.polaris.credential", f"{engineer_principal.credentials.client_id}:{engineer_principal.credentials.client_secret}")

  # Set the warehouse to the name of the catalog we created
  .config("spark.sql.catalog.polaris.warehouse", catalog_name)

  # Scope set to PRINCIPAL_ROLE:ALL
  .config("spark.sql.catalog.polaris.scope", 'PRINCIPAL_ROLE:ALL')

  # Enable access credential delegation
  .config("spark.sql.catalog.polaris.header.X-Iceberg-Access-Delegation", 'vended-credentials')

  .config("spark.sql.catalog.polaris.io-impl", "org.apache.iceberg.io.ResolvingFileIO")
  .config("spark.sql.catalog.polaris.s3.region", "us-west-2")
  .config("spark.history.fs.logDirectory", "/home/iceberg/spark-events")).getOrCreate()


# USE polaris
Tell Spark to use the Polaris catalog

In [9]:
spark.sql("USE polaris")
spark.sql("SHOW NAMESPACES").show()

+---------+
|namespace|
+---------+
+---------+



# Create Nested Namespaces

In [10]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS " + namespace)

DataFrame[]

# Create a table

In [11]:
spark.sql("USE " + namespace)
spark.sql("CREATE or replace TABLE tbl1 (col1 integer, col2 string) using iceberg TBLPROPERTIES('_source'='delta') location 'file:///tmp/polaris/ForeignTableDemo/'")

DataFrame[]

# It's Empty

In [12]:
spark.sql("show tables").show()

+----------------+---------+-----------+
|       namespace|tableName|isTemporary|
+----------------+---------+-----------+
|ForeignTableDemo|     tbl1|      false|
+----------------+---------+-----------+



In [13]:
spark.sql("select * from tbl1").show()

+---+-------+---+----+-------------------+
| id|   name|age|city|          create_ts|
+---+-------+---+----+-------------------+
|  3|Michael| 35| ORD|2023-09-28 00:00:00|
|  2|  Emily| 30| SFO|2023-09-28 00:00:00|
|  6|Charlie| 31| DFW|2023-08-29 00:00:00|
|  5|    Bob| 28| SEA|2023-09-23 00:00:00|
|  1|   John| 25| NYC|2023-09-28 00:00:00|
|  4| Andrew| 40| NYC|2023-10-28 00:00:00|
+---+-------+---+----+-------------------+



# Insert some records from Delta notebook and then refresh and query
* Querying again shows some records

In [14]:
spark.sql("refresh table tbl1")
spark.sql("select * from tbl1").show()

+---+-------+---+----+-------------------+
| id|   name|age|city|          create_ts|
+---+-------+---+----+-------------------+
|  7|  Alice| 25| NYC|2023-09-28 00:00:00|
|  8|    Tim| 30| SFO|2023-09-28 00:00:00|
| 10|    Bob| 40| SFO|2023-10-28 00:00:00|
|  9|   Andy| 35| ORD|2023-09-28 00:00:00|
|  3|Michael| 35| ORD|2023-09-28 00:00:00|
|  2|  Emily| 30| SFO|2023-09-28 00:00:00|
|  6|Charlie| 31| DFW|2023-08-29 00:00:00|
|  5|    Bob| 28| SEA|2023-09-23 00:00:00|
|  1|   John| 25| NYC|2023-09-28 00:00:00|
|  4| Andrew| 40| NYC|2023-10-28 00:00:00|
+---+-------+---+----+-------------------+



# Clean up the data

In [15]:
spark.sql("drop table tbl1")
spark.sql("drop namespace ForeignTableDemo")